### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from scripts.AtualizaUltimosJogos import *
import warnings
warnings.filterwarnings('ignore')

### Pegando os jogos do dia

In [2]:
# Obter a data de hoje
hoje = date.today()
ontem = hoje - timedelta(days=1)

### Atualizando os datasets que tem jogo hoje

In [3]:
dia = hoje

In [4]:
# Pegando os jogos do dia para atualizar
jogos_passados = pd.read_csv(f'jogos_do_dia/{dia}.csv')
dia_jogos = jogos_passados['Date'].iloc[0]
id_jogos = jogos_passados['Fixture ID'].unique().tolist()
print(f'Jogos do dia {dia_jogos}\n')

# Buscando os resultados
results = atualiza_ultimos_jogos(id_jogos = id_jogos)

# Merjando os resultados no dataset de jogos do dia
df_resultados = pd.merge(left=jogos_passados, right=results[['Fixture ID', 'Home_Pts', 'Away_Pts', 'Status']], on='Fixture ID', how='left')
df_resultados['Status'].replace(np.nan, 'Não iniciado', inplace=True)

# Organizando as colunas do df_resultados
df_resultados = df_resultados[['Fixture ID', 'Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', 'Status']]
df_resultados = df_resultados.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
df_resultados.sort_values('Time', inplace=True)
df_resultados.reset_index(inplace=True, drop=True)

Jogos do dia 2023-11-08



100%|██████████| 105/105 [02:09<00:00,  1.23s/it]


In [5]:
# Criando df com jogos que ainda não terminaram
df_restantes = df_resultados[(df_resultados['Status'] != 'Após Tempo Extra') & (df_resultados['Status'] != 'Encerrado') & (df_resultados['Status'] != 'Após Pênaltis')]
df_restantes = df_restantes[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Status']]

# Limpando o df_resultados
df_resultados = df_resultados[(df_resultados['Status'] == 'Após Tempo Extra') | (df_resultados['Status'] == 'Encerrado') | (df_resultados['Status'] == 'Após Pênaltis')]
df_resultados = df_resultados[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A']]

# Exclui os registros que já existem no dataset da temporada
temporada = pd.read_csv('data/Entire_Season.csv')
n_antes = len(temporada)
temporada = pd.concat([temporada, df_resultados], ignore_index=True)
temporada = temporada.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
temporada.reset_index(drop=True, inplace=True)
temporada.to_csv('data/Entire_Season.csv', index=False)

print(f'Dataset de temporada atualizado com {len(temporada) - n_antes} jogos.\n')
print('----------------------------------------------------------\n')
print(f'Atenção! {len(df_restantes)} jogos não finalizados.\n')
display(df_restantes)

Dataset de temporada atualizado com 89 jogos.

----------------------------------------------------------

Atenção! 16 jogos não finalizados.



,Date,League,Time,Home,Away,Home_Pts,Away_Pts,Status
80,2023-11-08,Uruguai Liga Uruguaia,21:00,Nacional,Malvin,36.0,31.0,Abandon.
90,2023-11-08,Eua Ncaa,21:00,Loyola (Ill).,Florida Atlantic,NaN,NaN,Não iniciado
91,2023-11-08,Eua Nba,21:30,New York Knicks,San Antonio Spurs,101.0,79.0,4º Quarto2'
92,2023-11-08,Eua Nba,21:30,Brooklyn Nets,Los Angeles Clippers,89.0,79.0,4º Quarto6'
93,2023-11-08,Canadá U Sports,22:00,TMU,McMaster,NaN,NaN,Não iniciado
94,2023-11-08,Canadá U Sports,22:00,Guelph,Waterloo,NaN,NaN,Não iniciado
95,2023-11-08,Eua Nba,22:00,Memphis Grizzlies,Miami Heat,53.0,59.0,3º Quarto2'
96,2023-11-08,Eua Nba,22:00,Houston Rockets,Los Angeles Lakers,66.0,45.0,Intervalo
97,2023-11-08,Eua Nba,22:00,Milwaukee Bucks,Detroit Pistons,69.0,56.0,3º Quarto2'
98,2023-11-08,Eua Nba,22:00,Minnesota Timberwolves,New Orleans Pelicans,84.0,55.0,3º Quarto6'
